In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/screener_sbanalysis_api/"
      ]

In [2]:
sys.path += paths

In [3]:
# !pip install elasticsearch-dsl

In [4]:
# !pip install psycopg2-binary

In [5]:

from pandas.io import sql as psql

In [6]:
from lib.static_db import PgsqlExecutor

In [7]:
snpsource_write = {
        "host": "snp.cluster-csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [8]:
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [9]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "postgres",
        "credentials": "DqR567Q1",
        "database": "modelresearch"
      }

In [68]:
db = PgsqlExecutor(snpsource)

In [101]:
sql = """
SELECT distinct tickersymbol, tradingitemid, exchangesymbol as symbol  from v2mv_sbt_company 
    where is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA' and tickersymbol = 'AAPL'
"""
data = db.fetch_all_rows(sql)
print(data.row_count)

1


In [102]:
data.query_data

[Record(tickersymbol='AAPL', tradingitemid=2590360, symbol='NasdaqGS')]

In [100]:
type(data.query_data[0].tradingitemid)

int

In [92]:
missing_data_symbol = []

sql = """
select distinct tickersymbol as symbol from
v2mv_sbt_company 
    where is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA'
"""
db_research = PgsqlExecutor(research_db)

data = db.fetch_all_rows(sql)
for row in data.query_data:
    missing_data_symbol.append(row.symbol)

In [69]:
missing_data_symbol = []

sql = """
select distinct symbol from dcf_values where is_active = 0 and fiscal_year = 2019
"""
db_research = PgsqlExecutor(research_db)

data = db_research.fetch_all_rows(sql)
for row in data.query_data:
    missing_data_symbol.append(row.symbol)

In [70]:
data.row_count

1132

In [93]:
sql = """
select exchangesymbol, count(*) from v2mv_sbt_company where tickersymbol in %s and exchangecountry = 'USA'
group by 1
"""
data = db.fetch_all_rows(sql, args=[tuple(missing_data_symbol)])

In [88]:
# sql = """
# select exchangesymbol , count(*) FROM v2mv_sbt_company 
#     where is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
#     and securityname ilike 'common%stock' 
#              and exchangecountry = 'USA'
#      GROUP BY exchangesymbol
                      
# """
# data = db.fetch_all_rows(sql)

In [94]:
count = 0
for row in data.query_data:
    print (row)
    count += row.count
    

print(count)

Record(exchangesymbol='AMEX', count=128)
Record(exchangesymbol='ARCA', count=90)
Record(exchangesymbol='BATS', count=10)
Record(exchangesymbol='NasdaqCM', count=534)
Record(exchangesymbol='NasdaqGM', count=297)
Record(exchangesymbol='NasdaqGS', count=952)
Record(exchangesymbol='NYSE', count=1123)
Record(exchangesymbol='OTCBB', count=1)
Record(exchangesymbol='OTCPK', count=3840)
Record(exchangesymbol='XNYS', count=2)
6977


In [10]:
sql = """

select tradingitemid from mv_sbt_company where tickersymbol = '{symbol}' and exchangesymbol ilike  'NASDAQ%'
limit 1
""".format(symbol="ABCB")

data = db.fetch_one_row(sql)
print (data.query_data.tradingitemid)
tradingitemid = data.query_data.tradingitemid

25082102


In [21]:
sql = """

select distinct tickersymbol from mv_sbt_company where  companytype = 'Public Company' and country='USA'
""".format(symbol="ABCB")

data = db.fetch_one_row(sql)
print (data.query_data.tickersymbol)
print (data.row_count)

#92574, 3233, 2727, 71629, 13546 v2mv_sbt_company


QMCO
13546


In [22]:
sql = """

select * from v2mv_sbt_company limit 10
""".format(symbol="ABCB")

data = db.fetch_one_row(sql)
print (data.query_data)
print (data.row_count)

#92574, 3233, 2727, 71629, 13546 v2mv_sbt_company


Record(companyid=18511, companyname='3i Group plc', ceo='Simon A. Borrows', companytype='Public Investment Firm', ciqcompanystatustype='Operating', city='London', country='GBR', state='Greater London', incorporationcountry='GBR', incorporationstate=None, officefaxvalue='44 20 7975 3232', officephonevalue='44 20 7975 3131', otherphonevalue=None, streetaddress='16 Palace Street', streetaddress2=None, streetaddress3=None, streetaddress4=None, yearfounded=1945, monthfounded=None, dayfounded=None, zipcode='SW1E 5JD', webpage='www.3i.com', securityname='Ordinary Shares', securityfeaturename=None, exchangesymbol='OTCPK', exchangename='Pink Sheets LLC', exchangecurrency='USD', tickersymbol='TGOP.F', tradingitemid=23515138, identifiervalue=None, identifiertypename=None, gicssector=None, gicsgroup=None, gicsindustry=None, gicssubindustry=None, businessdescription='3i Group plc is a private equity firm specializing in mature companies, growth capital, middle markets, infrastructure, and managemen

In [30]:
sql = """

select * from mv_sbt_company

where tickersymbol = 'AAGH'
"""

data = db.fetch_one_row(sql, args=[tradingitemid])
print (data.query_data)
company_id = data.query_data.companyid

Record(companyid=252558709, companyname='America Great Health', companytype='Public Company', ciqcompanystatustype='Operating', ceo='Mike Q. Wang', city='Alhambra', country='USA', state='California', incorporationcountry='USA', incorporationstate='Wyoming', officefaxvalue=None, officephonevalue='626-576-1299', otherphonevalue=None, streetaddress='1609 West Valley Boulevard', streetaddress2='Unit 338A', streetaddress3=None, streetaddress4=None, yearfounded=None, monthfounded=None, dayfounded=None, zipcode='91803', webpage='www.cwnmusa.com', securityname='Common Stock', securityfeaturename=None, exchangesymbol='OTCPK', exchangename='Pink Sheets LLC', exchangecountry='USA', exchangecurrency='USD', is_primary_exchange=True, tickersymbol='AAGH', tradingitemid=209137990, identifiervalue=None, identifiertypename=None, gicssector=None, gicsgroup=None, gicsindustry=None, gicssubindustry=None, businessdescription='America Great Health does not have significant operations. The company focuses on 

In [35]:
sql = """
select distinct tickersymbol from v2mv_sbt_company 
    where is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA'
             and exchangesymbol not ilike 'OTC%'
"""
data = db.fetch_one_row(sql)
print (data.row_count)

3134


In [75]:
sql = """SELECT  priceopen, pricehigh, pricelow,  priceclose,  volume, 
    extract (epoch from pricingdate) timeseries, pricingdate
	    FROM public.v_price_timeseries 
	        WHERE tradingitemid = %s 
	        AND pricingdate between '2019-06-03'::date and now() order by timeseries;
"""
data = db.get_dataframe(sql, ( tradingitemid,))
data.head()

priceopen  pricehigh  pricelow  priceclose  volume    timeseries  \
0      35.26      36.25    34.980       35.90  309157  1.559520e+09   
1      36.49      37.30    36.420       37.25  261248  1.559606e+09   
2      37.21      37.42    36.310       37.03  276270  1.559693e+09   
3      37.00      37.01    36.330       36.90  167623  1.559779e+09   
4      36.83      37.00    36.535       36.58  140498  1.559866e+09   

  pricingdate  
0  2019-06-03  
1  2019-06-04  
2  2019-06-05  
3  2019-06-06  
4  2019-06-07

In [76]:
sql = """
SELECT marketcap, tev, sharesoutstanding from v_snapshot_market_cap where companyid = %s
"""
data = db.get_dataframe(sql, ( company_id,))
data.head()

marketcap   tev  sharesoutstanding
0  1642.082663  None           69256966

In [43]:
# mv_sbt_company
valid_symbols = []
sql = """
SELECT distinct tickersymbol  from mv_sbt_company where companytype = 'Public Company' 
and exchangesymbol ilike 'NASDAQ%' 
"""
data = db.fetch_all_rows(sql)
for row in data.query_data:
    valid_symbols.append(row.tickersymbol)
    
print(valid_symbols[0:10])

['AACG', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AAWW', 'AAXN', 'ABCB', 'ABEO']


In [44]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "postgres",
        "credentials": "DqR567Q1",
        "database": "modelresearch"
      }

In [46]:
# research_db_connected = PgsqlExecutor(research_db)

In [47]:
# sql = """
# select * from dcf_values where symbol not in %s

# """
# data = research_db_connected.fetch_all_rows(sql, args=[tuple(valid_symbols)])
# print(data.row_count)

# for row in data.query_data:
#     print(row)

803
Record(id=1985, symbol='AAXJ', is_active=0, current_price=66.09, market_growth_rate=0.095, current_growth_rate=None, fiscal_year=None, market_dcf=None, dcf=None, error='run failed: AAXJ local variable \'cash_interest_paid_value\' referenced before assignmentTraceback (most recent call last):\n  File "C:/Users/tbui/Documents/Stansberry-Terminal-MkI/datafactory/models/dcf_model.py", line 222, in calculate_dcf\n    long_term_debt_values = long_term_debt["dataitemvalue"].iloc[-1] * long_term_debt["unittypevalue"].iloc[-1]\n  File "C:\\Users\\tbui\\PycharmProjects\\terminal-mkl\\venv\\lib\\site-packages\\pandas\\core\\indexing.py", line 1373, in __getitem__\n    return self._getitem_axis(maybe_callable, axis=axis)\n  File "C:\\Users\\tbui\\PycharmProjects\\terminal-mkl\\venv\\lib\\site-packages\\pandas\\core\\indexing.py", line 1830, in _getitem_axis\n    self._is_valid_integer(key, axis)\nIndexError: single positional indexer is out-of-bounds\n\nDuring handling of the above exception, 

In [48]:
# sql = """
# delete from dcf_values where symbol not in %s

# """
# data = research_db_connected.modify_rows(sql, args=[tuple(valid_symbols)])
# print(data.row_count)



803


In [78]:
dataitemid = 4423

sql = """


        WITH fiscalYear as (
        select * from ciqlatestinstancefinperiod where companyid = %s 
        AND fiscalyear >= EXTRACT(YEAR FROM now()) - 10 order by fiscalyear
        ),
        itemName as (select dataitemid, dataitemname from 
            ciqdataitem 
            where dataitemid = %s
            )
        
        SELECT ti.tradingitemid,
            fd.dataitemvalue,
            fd.dataitemid,
            fd.financialperiodid,
            curr.isocode,
            c.companyid,
            itemName.dataitemname,
            unit.unittypevalue,
            fp.latestperiodflag,
            fiscalYear.fiscalyear
            
           FROM ciqcompany c
             JOIN ciqsecurity s ON c.companyid = s.companyid
             JOIN ciqtradingitem ti ON ti.securityid = s.securityid
             JOIN ciqlatestinstancefinperiod fp ON fp.companyid = c.companyid
             JOIN ciqperiodtype pt ON pt.periodtypeid = fp.periodtypeid
             JOIN ciqfinancialdata fd ON fd.financialperiodid = fp.financialperiodid
             JOIN ciqdataitem di ON di.dataitemid = fd.dataitemid
             JOIN ciqcurrency curr ON curr.currencyid = fp.currencyid
             JOIN fiscalYear ON fd.financialperiodid = fiscalYear.financialperiodid
             JOIN itemName on itemName.dataitemid = fd.dataitemid
             JOIN ciqfinunittype unit on unit.unittypeid = fd.unittypeid
          WHERE fp.periodtypeid = 1 AND c.companyid = %s 
          AND fd.dataitemid = %s
          
          ORDER BY fiscalYear.fiscalyear;

"""
data = db.get_dataframe(sql, ( company_id, dataitemid, company_id,dataitemid,))
data.head()

Empty DataFrame
Columns: [tradingitemid, dataitemvalue, dataitemid, financialperiodid, isocode, companyid, dataitemname, unittypevalue, latestperiodflag, fiscalyear]
Index: []

In [79]:
from datetime import datetime
now = datetime.now()

In [80]:
now.replace(year=2018)

datetime.datetime(2018, 6, 3, 16, 41, 0, 513230)

In [21]:
sql = "select * from v_price_timeseries limit 10"
data = db.get_dataframe(sql)
data.head()

tradingitemid pricingdate  priceopen  pricehigh  pricelow  pricemid  \
0       20239152  2002-10-21        0.0        0.0       0.0    1.0025   
1       20239152  2002-10-22        0.0        0.0       0.0    1.0145   
2       20239152  2002-10-23        0.0        0.0       0.0    1.0370   
3       20239152  2002-10-24        0.0        0.0       0.0    1.0575   
4       20239152  2002-10-25        0.0        0.0       0.0    1.0660   

   priceclose  pricebid  priceask   volume  adjustmentfactor  vwap  \
0       1.000     1.000     1.005   240800              10.0   0.0   
1       1.005     1.005     1.024  2926350              10.0   0.0   
2       1.035     1.035     1.039   261640              10.0   0.0   
3       1.070     1.055     1.060   411110              10.0   0.0   
4       1.068     1.064     1.068   592730              10.0   0.0   

   pacvertofeedpop exchangecurrency  asdivadjfactor  divadjprice  
0              561              EUR        0.668471     0.668471  
1              561              EUR        0.668471     0.671814  
2              561              EUR        0.668471     0.691868  
3              561              EUR        0.668471     0.715264  
4              561              EUR        0.668471     0.713927

In [32]:


sql = "select * from v2mv_sbt_company limit 1"
data = db.get_dataframe(sql)
data.head()

companyid   companyname               ceo             companytype  \
0      18511  3i Group plc  Simon A. Borrows  Public Investment Firm   

  ciqcompanystatustype    city country           state incorporationcountry  \
0            Operating  London     GBR  Greater London                  GBR   

  incorporationstate  ... gicssector gicsgroup gicsindustry gicssubindustry  \
0               None  ...       None      None         None            None   

                                 businessdescription longbusinessdescription  \
0  3i Group plc is a private equity firm speciali...                    None   

  employeecount  templatetypename exchangecountry is_primary_exchange  
0         240.0          Standard             USA               False  

[1 rows x 41 columns]

In [10]:
write_db = PgsqlExecutor(snpsource_write)

sql = """

CREATE OR REPLACE VIEW public.v_napshot_dcf_financial
AS
                SELECT ti.tradingitemid,
            fd.dataitemvalue,
            fd.dataitemid,
            fd.financialperiodid,
            c.companyid,
            unit.unittypevalue,
            fp.latestperiodflag,
            fp.fiscalyear
            
                 FROM ciqcompany c
                JOIN ciqsecurity s ON c.companyid = s.companyid
                JOIN ciqtradingitem ti ON ti.securityid = s.securityid
                JOIN ciqlatestinstancefinperiod fp ON fp.companyid = c.companyid 
                 JOIN ciqperiodtype pt ON pt.periodtypeid = fp.periodtypeid
                JOIN ciqfinancialdata fd ON fd.financialperiodid = fp.financialperiodid
                JOIN ciqdataitem di ON di.dataitemid = fd.dataitemid
                JOIN ciqcurrency curr ON curr.currencyid = fp.currencyid          
                 JOIN ciqfinunittype unit on unit.unittypeid = fd.unittypeid
                WHERE fp.periodtypeid = 1 
                                AND fp.fiscalyear >= EXTRACT(YEAR FROM now()) - 25
                                AND fd.dataitemid in (1193, 4423, 3028, 3069, 4422, 1014)
                                
                                 
                                ;
                
ALTER TABLE public.V_napshot_dcf_financial OWNER to snpx_read;
 
GRANT ALL ON TABLE public.V_napshot_dcf_financial to snpx_read;

"""
write_db.modify_rows(sql)

ExecutionResults(query_data=None, row_count=-1, cursor_description=None)

In [9]:
print ("Item financial Income: **ciqdataitemfinancial**")
sql = """
select * from v_snapshot_market_cap where companyid = 24937
"""
data = db.get_dataframe(sql)
data.head()

Item financial Income: **ciqdataitemfinancial**


companyid     marketcap pricingdate           tev  sharesoutstanding
0      24937  1.395006e+06  2020-06-01  1.419716e+06         4334335000

Item financial Income: **ciqdataitemfinancial**


In [39]:
sql = """


        WITH fiscalYear as (
        select * from ciqlatestinstancefinperiod where companyid = %s 
        AND fiscalyear >= EXTRACT(YEAR FROM now()) - 5 order by fiscalyear
        ),
        itemName as (select dataitemid, dataitemname from 
            ciqdataitem 
            where dataitemid = %s
            )
        
        SELECT ti.tradingitemid,
            fd.dataitemvalue,
            fd.dataitemid,
            fd.financialperiodid,
            curr.isocode,
            c.companyid,
            itemName.dataitemname,
            unit.unittypevalue,
            fp.latestperiodflag,
            fiscalYear.fiscalyear
            
           FROM ciqcompany c
             JOIN ciqsecurity s ON c.companyid = s.companyid
             JOIN ciqtradingitem ti ON ti.securityid = s.securityid
             JOIN ciqlatestinstancefinperiod fp ON fp.companyid = c.companyid
             JOIN ciqperiodtype pt ON pt.periodtypeid = fp.periodtypeid
             JOIN ciqfinancialdata fd ON fd.financialperiodid = fp.financialperiodid
             JOIN ciqdataitem di ON di.dataitemid = fd.dataitemid
             JOIN ciqcurrency curr ON curr.currencyid = fp.currencyid
             JOIN fiscalYear ON fd.financialperiodid = fiscalYear.financialperiodid
             JOIN itemName on itemName.dataitemid = fd.dataitemid
             JOIN ciqfinunittype unit on unit.unittypeid = fd.unittypeid
          WHERE fp.periodtypeid = 1 AND c.companyid = %s 
          AND fd.dataitemid = %s
          
          ORDER BY fiscalYear.fiscalyear;

"""
data = db.get_dataframe(sql, (18749, 2588568, 18749, 2588568,))
data.head()


Empty DataFrame
Columns: [tradingitemid, dataitemvalue, dataitemid, financialperiodid, isocode, companyid, dataitemname, unittypevalue, latestperiodflag, fiscalyear]
Index: []

In [36]:
sql = """

select c.companyid from
ciqcompany c
             JOIN ciqsecurity s ON c.companyid = s.companyid
             JOIN ciqtradingitem ti ON ti.securityid = s.securityid
			 where ti.tradingitemid = %s
"""
data = db.get_dataframe(sql, (2588568,))
data.head()



companyid
0      18749

In [35]:

sql= """
select tradingitemid from mv_sbt_company where tickersymbol = '{symbol}' and exchangesymbol ilike  'NASDAQ%'
limit 1
""".format(symbol="AMZN")
data = db.get_dataframe(sql)
data.head()

tradingitemid
0        2588568

In [10]:
sql= """
select distinct tickersymbol from mv_sbt_company where exchangesymbol ilike  'NASDAQ%'
""".format(symbol="AMZN")
data = db.get_dataframe(sql)
data.head()

tickersymbol
0         AACG
1          AAL
2         AAME
3         AAOI
4         AAON

In [12]:
sql= """
select distinct exchangesymbol from mv_sbt_company where tickersymbol = 'AAPL'
""".format(symbol="AMZN")
data = db.get_dataframe(sql)
data.head()

exchangesymbol
0           BASE
1            BVC
2            BVL
3       NasdaqGS
4           SNSE

In [18]:
sql= """
select distinct exchangesymbol from mv_sbt_company 
""".format(symbol="AMZN")
data = db.get_dataframe(sql)
for row in data["exchangesymbol"].iteritems():
    print (row)

(0, 'BOIN')
(1, 'SASE')
(2, 'MCXX')
(3, 'TTSE')
(4, 'DAR')
(5, 'XSSC')
(6, 'BRVM')
(7, 'NYSE')
(8, 'UNQ-USA')
(9, 'F-CHE')
(10, 'BDM')
(11, 'SHSC')
(12, 'MutualFund')
(13, 'SET')
(14, 'CNSX')
(15, 'PLSE')
(16, 'TSEC')
(17, 'CSE')
(18, 'DSE')
(19, 'ENXTBR')
(20, 'DB')
(21, 'UKR')
(22, 'MUSE')
(23, 'ADX')
(24, 'CPSE')
(25, 'DUSE')
(26, 'CASE')
(27, 'DFM')
(28, 'BUSE')
(29, 'F-ITA')
(30, 'SEHK')
(31, 'OM')
(32, 'LJSE')
(33, 'ZMSE')
(34, 'BDB')
(35, 'TSE')
(36, 'GHSE')
(37, 'KOSE')
(38, 'HNX')
(39, 'NEOE')
(40, 'XQMH')
(41, 'F-DEU')
(42, 'NGSE')
(43, 'SKOPJE')
(44, 'WBAG')
(45, 'AMEX')
(46, 'BITE')
(47, 'HOSE')
(48, 'TLSE')
(49, 'BME')
(50, 'SNSE')
(51, 'NGM')
(52, 'COSE')
(53, 'MTSE')
(54, 'BDL')
(55, 'ENXTAM')
(56, 'BAX')
(57, 'NSEI')
(58, 'ASE')
(59, 'BER')
(60, 'MICEX')
(61, 'SOMA')
(62, 'OTCBB')
(63, 'XNAS')
(64, 'UX')
(65, 'OTCUS')
(66, 'BSE')
(67, 'Catalist')
(68, 'HMSE')
(69, 'COBL')
(70, 'F-MEX')
(71, 'NASE')
(72, 'ATSE')
(73, 'BRSE')
(74, 'XTRA')
(75, 'TSX')
(76, 'OFEX')
(77, 'UN

In [13]:
print("Financial Instance Type : **ciqfininstancetype**")
sql = """
select * from ciqfininstancetype
"""
data = db.get_dataframe(sql)
data.head()

Financial Instance Type : **ciqfininstancetype**


instancetypeid                           instancetypename  \
0               1                              Press Release   
1               2                          Original Instance   
2               3                                     Encore   
3               4         Backward column of a press release   
4               5  Q4-Calculated using more than 1Yr old YTD   

  instancetypeabbreviation  
0                        P  
1                        O  
2                        E  
3                        X  
4                        I

In [14]:
print ("financial collection type: **ciqfindatacollectiontype**")
sql = """
select * from ciqfindatacollectiontype
"""
data = db.get_dataframe(sql)
data.head()

financial collection type: **ciqfindatacollectiontype**


datacollectiontypeid datacollectiontypename
0                     1       Income Statement
1                     2          Balance Sheet
2                     3              Cash Flow
3                     4          Non-Financial
4                     5            TEV/Capital

In [15]:
print ("Item financial Income: **ciqdataitemfinancial**")
sql = """
select * from ciqdataitemfinancial where dataitemid in (28, 15, 100063, 100066, 4051, 4017, 21624, 112991)
"""
data = db.get_dataframe(sql)
data.head()

Item financial Income: **ciqdataitemfinancial**


dataitemid  datacollectiontypeid
0          15                     1
1          28                     1
2        4017                    53
3        4051                     1
4      100063                    53

In [16]:
print ("Item financial Non-Financial")
sql = """
with finType as (
select datacollectiontypeid from ciqfindatacollectiontype where datacollectiontypename = 'Non-Financial'
)

select * from ciqdataitem where dataitemid in (select 
dataitemid from ciqdataitemfinancial where  datacollectiontypeid in (select datacollectiontypeid from finType)
)
"""
data = db.get_dataframe(sql)
data.head()

Item financial Non-Financial


dataitemid                     dataitemname             dataitemdescription
0        5002                  Auditor Opinion                            None
1       30889                          T to lb  Metric Ton to Pound - (2204.6)
2        5001                     Auditor Name                            None
3        2151     Calculation Type - Cash Flow                            None
4        1303  Restatement Type, Balance Sheet                            None

In [17]:
print ("Item financial TEV/Capital")
sql = """
with finType as (
select datacollectiontypeid from ciqfindatacollectiontype where datacollectiontypename = 'TEV/Capital'
)

select * from ciqdataitem where dataitemid in (select 
dataitemid from ciqdataitemfinancial where  datacollectiontypeid in (select datacollectiontypeid from finType)
)
"""
data = db.get_dataframe(sql)
data.head()

Item financial TEV/Capital


dataitemid                               dataitemname  \
0        4438          TEV Less Market Cap - (Insurance)   
1        1005                     Total Preferred Equity   
2        1002      Total Cash And Short Term Investments   
3        1312  Minority Interest, Total (Incl. Fin. Div)   
4        4364                                   Net Debt   

                                 dataitemdescription  
0                    TEV less Market Cap (Insurance)  
1  Total Preferred is defined as the total nomina...  
2  Total cash and Short Term Investments signify ...  
3                            Total Minority Interest  
4  Net Debt is the difference between total debt ...

In [20]:
print ("Item financial Balance Sheet: **ciqdataitemfinancial**")
sql = """
with finType as (
select datacollectiontypeid from ciqfindatacollectiontype where datacollectiontypename = 'Balance Sheet'
)

select * from ciqdataitem where dataitemid in (select 
dataitemid from ciqdataitemfinancial where  datacollectiontypeid in (select datacollectiontypeid from finType)
) 
order by dataitemid
"""
data = db.get_dataframe(sql)
data.head(50)

Item financial Balance Sheet: **ciqdataitemfinancial**


dataitemid                                       dataitemname  \
0         1001                                  Total Receivables   
1         1003                                          Net Loans   
2         1004                   Net Property Plant And Equipment   
3         1007                                       Total Assets   
4         1008                               Total Current Assets   
5         1009                          Total Current Liabilities   
6         1010                                     Total Deposits   
7         1011                                  Total investments   
8         1012            Total Liabilities - (Template Specific)   
9         1013                       Total Liabilities And Equity   
10        1014                        Total Long-Term Liabilities   
11        1015                           Total Real Estate Assets   
12        1016                            Accrued Expenses, Total   
13        1017      Accrued Expenses, Total - (Template Specific)   
14        1018                            Accounts Payable, Total   
15        1019      Accounts Payable, Total - (Template Specific)   
16        1020   Accounts Receivable, Total - (Template Specific)   
17        1021                         Accounts Receivable, Total   
18        1022  Comprehensive Income and Other - (Template Spe...   
19        1023      Current Portion of Long-Term Debt (Collected)   
20        1024  Current Portion of Long-Term Debt - (Template ...   
21        1025                         Deferred Charges Long-Term   
22        1026                      Deferred Tax Assets Long-Term   
23        1027                 Deferred Tax Liability Non Current   
24        1028                     Comprehensive Income and Other   
25        1029       Finance Division Other Current Assets, Total   
26        1030                      Finance Division Debt Current   
27        1031  Finance Division Other Current Liabilities, Total   
28        1032          Finance Division Loans and Leases Current   
29        1033        Finance Division Loans and Leases Long-Term   
30        1034     Finance Division Other Long-Term Assets, Total   
31        1035                  Finance Division Debt Non Current   
32        1036  Finance Division Other Non Current Liabilities...   
33        1037                  Insurance And Annuity Liabilities   
34        1038              Long-Term Investments - (FS Template)   
35        1039  Investment In Equity and Preferred Securities,...   
36        1040                           Other Intangibles, Total   
37        1041     Other Intangibles, Total - (Template Specific)   
38        1042                               Inventories - Others   
39        1043                                          Inventory   
40        1044                       Investment Securities, Total   
41        1045        Loans and Lease Receivables - (FS Template)   
42        1046                              Short-term Borrowings   
43        1047        Short-Term Borrowings - (Template Specific)   
44        1048                                   Notes Receivable   
45        1049                                     Long-Term Debt   
46        1050                         Loans Receivable Long-Term   
47        1051                         Mortgage Backed Securities   
48        1052                                  Minority Interest   
49        1053     Minority Interest, Total - (Template Specific)   

                                  dataitemdescription  
0   This item represents the sum of:\r\n\r\n   1. ...  
1   This item represents loans made less allowance...  
2   This item represents gross property, plant and...  
3   Total Assets as an acquirer security represent...  
4   This item represents assets which are expected...  
5   This item represents the sub total of all the ...  
6   This item represents the sum of all domestic a...  
7   This item represents the sum of 

In [10]:
print ("Item financial  **Cash Flow**")
sql = """
with finType as (
select datacollectiontypeid from ciqfindatacollectiontype
)

select * from ciqdataitem where dataitemid in (select 
dataitemid from ciqdataitemfinancial where  datacollectiontypeid in (select datacollectiontypeid from finType)
)

and dataitemid in (4422, 4423, 1007, 1008, 1009, 1047, 1193, 3069, 3028)
order by dataitemid
"""
data = db.get_dataframe(sql)
data.head(50)

Item financial  **Cash Flow**


dataitemid                                 dataitemname  \
0        1007                                 Total Assets   
1        1008                         Total Current Assets   
2        1009                    Total Current Liabilities   
3        1047  Short-Term Borrowings - (Template Specific)   
4        1193         Long-Term Debt - (Template Specific)   
5        3028                           Cash Interest Paid   
6        3069             Effective Tax Rate - (Collected)   
7        4422                       Levered Free Cash Flow   
8        4423                     Unlevered Free Cash Flow   

                                 dataitemdescription  
0  Total Assets as an acquirer security represent...  
1  This item represents assets which are expected...  
2  This item represents the sub total of all the ...  
3  This item represents loan amounts for Banking ...  
4  This item represents loan amounts, which are t...  
5  This item represents cash outflows towards int...  
6  This item represents actual tax rate derived a...  
7                             Levered Free Cash Flow  
8                           Unlevered Free Cash Flow

In [20]:
print ("Item financial Income: **ciqdataitemfinancial**")
sql = """
with finType as (
select datacollectiontypeid from ciqfindatacollectiontype where datacollectiontypename = 'Income Statement'
)

select * from ciqdataitem where dataitemid in (select 
dataitemid from ciqdataitemfinancial where  datacollectiontypeid in (select datacollectiontypeid from finType)
)
"""
data = db.get_dataframe(sql)
data.head(50)

Item financial Income: **ciqdataitemfinancial**
Item financial Income: **ciqdataitemfinancial**


dataitemid                                       dataitemname  \
0          251  Non Operating Income (Expenses) - (Expense Block)   
1        22531                              Ceded Earned Premiums   
2        30044                               Cash Cost (T) - Coal   
3        32780              Writedowns, Diluted EPS Effect - (CS)   
4        30832                                 Net Operated Wells   
5        30452  Average Realized Price (T) After Treatment and...   
6          264                          Other Non Interest Income   
7        24887                                  Average Loan Size   
8        31164  Interest Income on Balances Due From Depositor...   
9        31252  Income on Sale of Mutual Funds, Domestic - (FF...   
10       31199                 Other Noninterest Income - (FFIEC)   
11       31200                 Total Noninterest Income - (FFIEC)   
12       30806                       Daily Production Growth, Gas   
13       32325  Loans To U.S. Depository Institutions - Charge...   
14       32328  Commercial and Industrial Loans To U.S. Addres...   
15       26560                                         Other Beds   
16       34407                         Other EPRA NAV Adjustments   
17         276                Property & Casualty Premiums Earned   
18       44064                         Operating Revenues - (CSD)   
19       45865              + Staff - Restructuring Costs - (CSD)   
20       45915  Def. Benefit Total Cost Incl. Supple Plans (OP...   
21        3323                        Total Proved Reserves - Gas   
22       23492                                 Fuel Cost / Gallon   
23       35087                                      Postpaid ARPU   
24       34215  Eliminations Arising From The Consolidation Of...   
25       25816                                    Racing Revenues   
26        3239  Stock Options Cancelled During The Year - Clas...   
27       46496     Amortization of Deferred Charges - (After Tax)   
28       22574  Net Claim & Claim Adjustment Expense Paid – Pr...   
29       32788  In Process R&D Expense, Diluted EPS Effect - (CS)   
30       24135                                 EBITDA (Excl. SBC)   
31       30738                     Cost of Capital - New Business   
32        3140                            Total Expense (Pension)   
33       22676                   Stock-Based Comp., Other (Total)   
34       30320  Average Realized Price (ct) Before Hedging - D...   
35       45166  D&A - Impairment Charges/(Reversals) (Addition...   
36       26420                    ASO Membership - (Managed Care)   
37       26650                    Square Feet Leased, Net - Total   
38       26434                                Selling Costs Ratio   
39       30856                Total Net Development Wells Drilled   
40          25             Interest Income, Total (Bank Template)   
41       40288                                Catastrophic Losses   
42         359  Provision For Loan Losses - (Bank / Brok. / FS...   
43       30800                  Production Growth, Oil Equivalent   
44       26620                         GLA Percent Leased - Total   
45       46101            Securitzed Interest (Additions) - (CSD)   
46          26        Non Interest Expense, Total (Bank Template)   
47        3319                            Other Adjustments - Gas   
48        3322                         Undeveloped Reserves - Gas   
49       24721                                      Finished Lots   

                                  dataitemdescription  
0   This item represents any income (expense), whi...  
1                                                None  
2                                  Mapped from TCCCMT  
3                                                 WDD  
4                                   Mapped from NOPWL  
5                              Mapped from TIARPATRMT  
6   This item represents the total of all residual...  
7                                   

dataitemid                                       dataitemname  \
0          251  Non Operating Income (Expenses) - (Expense Block)   
1        22531                              Ceded Earned Premiums   
2        30044                               Cash Cost (T) - Coal   
3        32780              Writedowns, Diluted EPS Effect - (CS)   
4        30832                                 Net Operated Wells   
5        30452  Average Realized Price (T) After Treatment and...   
6          264                          Other Non Interest Income   
7        24887                                  Average Loan Size   
8        31164  Interest Income on Balances Due From Depositor...   
9        31252  Income on Sale of Mutual Funds, Domestic - (FF...   
10       31199                 Other Noninterest Income - (FFIEC)   
11       31200                 Total Noninterest Income - (FFIEC)   
12       30806                       Daily Production Growth, Gas   
13       32325  Loans To U.S. Depository Institutions - Charge...   
14       32328  Commercial and Industrial Loans To U.S. Addres...   
15       26560                                         Other Beds   
16       34407                         Other EPRA NAV Adjustments   
17         276                Property & Casualty Premiums Earned   
18       44064                         Operating Revenues - (CSD)   
19       45865              + Staff - Restructuring Costs - (CSD)   
20       45915  Def. Benefit Total Cost Incl. Supple Plans (OP...   
21        3323                        Total Proved Reserves - Gas   
22       23492                                 Fuel Cost / Gallon   
23       35087                                      Postpaid ARPU   
24       34215  Eliminations Arising From The Consolidation Of...   
25       25816                                    Racing Revenues   
26        3239  Stock Options Cancelled During The Year - Clas...   
27       46496     Amortization of Deferred Charges - (After Tax)   
28       22574  Net Claim & Claim Adjustment Expense Paid – Pr...   
29       32788  In Process R&D Expense, Diluted EPS Effect - (CS)   
30       24135                                 EBITDA (Excl. SBC)   
31       30738                     Cost of Capital - New Business   
32        3140                            Total Expense (Pension)   
33       22676                   Stock-Based Comp., Other (Total)   
34       30320  Average Realized Price (ct) Before Hedging - D...   
35       45166  D&A - Impairment Charges/(Reversals) (Addition...   
36       26420                    ASO Membership - (Managed Care)   
37       26650                    Square Feet Leased, Net - Total   
38       26434                                Selling Costs Ratio   
39       30856                Total Net Development Wells Drilled   
40          25             Interest Income, Total (Bank Template)   
41       40288                                Catastrophic Losses   
42         359  Provision For Loan Losses - (Bank / Brok. / FS...   
43       30800                  Production Growth, Oil Equivalent   
44       26620                         GLA Percent Leased - Total   
45       46101            Securitzed Interest (Additions) - (CSD)   
46          26        Non Interest Expense, Total (Bank Template)   
47        3319                            Other Adjustments - Gas   
48        3322                         Undeveloped Reserves - Gas   
49       24721                                      Finished Lots   

                                  dataitemdescription  
0   This item represents any income (expense), whi...  
1                                                None  
2                                  Mapped from TCCCMT  
3                                                 WDD  
4                                   Mapped from NOPWL  
5                              Mapped from TIARPATRMT  
6   This item represents the total of all residual...  
7                                   

In [21]:
print ("Item financial Income: **ciqdataitemfinancial**")
sql = """
select * from ciqdataitemfinancial where datacollectiontypeid = 3
"""
data = db.get_dataframe(sql)
data.head()

Item financial Income: **ciqdataitemfinancial**
Item financial Income: **ciqdataitemfinancial**


dataitemid  datacollectiontypeid
0        2002                     3
1        2003                     3
2        2004                     3
3        2005                     3
4        2006                     3

dataitemid  datacollectiontypeid
0        2002                     3
1        2003                     3
2        2004                     3
3        2005                     3
4        2006                     3

In [22]:
"""
4051
100066
100063
15
28
"""
sql = """
select a.*, b.* from ciqdataitemfinancial a, ciqfindatacollectiontype b where a.datacollectiontypeid = b.datacollectiontypeid

and a.dataitemid in (4051,
100066,
100063,
15,
28)
""" 
data = db.get_dataframe(sql)
data.head()

dataitemid  datacollectiontypeid  datacollectiontypeid  \
0          15                     1                     1   
1          28                     1                     1   
2        4051                     1                     1   
3      100063                    53                    53   
4      100066                    53                    53   

  datacollectiontypename  
0       Income Statement  
1       Income Statement  
2       Income Statement  
3      IQ Added - Ratios  
4      IQ Added - Ratios

dataitemid  datacollectiontypeid  datacollectiontypeid  \
0          15                     1                     1   
1          28                     1                     1   
2        4051                     1                     1   
3      100063                    53                    53   
4      100066                    53                    53   

  datacollectiontypename  
0       Income Statement  
1       Income Statement  
2       Income Statement  
3      IQ Added - Ratios  
4      IQ Added - Ratios

In [23]:

sql = """
select * from v_snapshot_latest_financials where tradingitemid=2590360;
""" 
data = db.get_dataframe(sql)
data.head()



tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0        2590360   76477.000000        4051         1989628342     USD   
1        2590360      25.879730      100066         1989628342     USD   
2        2590360      17.537406      100063         1989628342     USD   
3        2590360   55256.000000          15         1989628342     USD   
4        2590360  260174.000000          28         1989628342     USD   

   companyid  
0      24937  
1      24937  
2      24937  
3      24937  
4      24937

tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0        2590360   76477.000000        4051         1989628342     USD   
1        2590360      25.879730      100066         1989628342     USD   
2        2590360      17.537406      100063         1989628342     USD   
3        2590360   55256.000000          15         1989628342     USD   
4        2590360  260174.000000          28         1989628342     USD   

   companyid  
0      24937  
1      24937  
2      24937  
3      24937  
4      24937

In [24]:

sql = """
select * from v_snapshot_market_cap where companyid=24937;
""" 
data = db.get_dataframe(sql)
data.head()



companyid     marketcap pricingdate           tev  sharesoutstanding
0      24937  1.383650e+06  2020-05-20  1.408360e+06         4334335000

companyid     marketcap pricingdate           tev  sharesoutstanding
0      24937  1.383650e+06  2020-05-20  1.408360e+06         4334335000

In [25]:
sql = """
select * from ciqfinperiod where financialperiodid = 1989628342;
""" 
data = db.get_dataframe(sql)
data.head()

financialperiodid  companyid  periodtypeid  calendaryear  calendarquarter  \
0         1989628342      24937             1          2019                3   

   fiscalyear  fiscalquarter  latestperiodflag  fiscalchainseriesid  
0        2019              4                 1                    1

financialperiodid  companyid  periodtypeid  calendaryear  calendarquarter  \
0         1989628342      24937             1          2019                3   

   fiscalyear  fiscalquarter  latestperiodflag  fiscalchainseriesid  
0        2019              4                 1                    1

In [26]:
print ("Unittype : **ciqfinunittype**")
sql = """
select * from ciqfinunittype limit 100
"""
data = db.get_dataframe(sql)
data.head()

Unittype : **ciqfinunittype**
Unittype : **ciqfinunittype**


unittypeid unittypename  unittypevalue
0           0       Actual              1
1           1    Thousands           1000
2           2     Millions        1000000

unittypeid unittypename  unittypevalue
0           0       Actual              1
1           1    Thousands           1000
2           2     Millions        1000000

In [27]:
import datetime
start_time = datetime.datetime.now()
print ("Financial collection data : **ciqlatestinstancefinperiod**")
sql = """
select * from ciqlatestinstancefinperiod  limit 10
"""
data = db.get_dataframe(sql)
data.head()


Financial collection data : **ciqlatestinstancefinperiod**
Financial collection data : **ciqlatestinstancefinperiod**


financialperiodid  companyid  periodtypeid periodenddate  calendaryear  \
0                  0     337346             3    1998-03-29          1998   
1                  1     397232             2    2000-12-31          2000   
2                  3      30666             2    2002-06-30          2002   
3                  4     288318             2    2003-03-31          2003   
4                  5     403623             3    2002-09-30          2002   

   calendarquarter  fiscalquarter  fiscalyear filingdate  currencyid  \
0                1              3        1998 1999-05-10         160   
1                4              4        2000 2002-06-28          21   
2                2              2        2002 2003-08-13         160   
3                1              1        2003 2004-05-07         160   
4                3              3        2002 2003-11-07          21   

   restatementtypeid  latestperiodflag  fiscalchainseriesid  
0                  3                 0                    1  
1                  3                 0                    1  
2                  3                 0                    1  
3                  5                 0                    1  
4                  3                 0                    1

financialperiodid  companyid  periodtypeid periodenddate  calendaryear  \
0                  0     337346             3    1998-03-29          1998   
1                  1     397232             2    2000-12-31          2000   
2                  3      30666             2    2002-06-30          2002   
3                  4     288318             2    2003-03-31          2003   
4                  5     403623             3    2002-09-30          2002   

   calendarquarter  fiscalquarter  fiscalyear filingdate  currencyid  \
0                1              3        1998 1999-05-10         160   
1                4              4        2000 2002-06-28          21   
2                2              2        2002 2003-08-13         160   
3                1              1        2003 2004-05-07         160   
4                3              3        2002 2003-11-07          21   

   restatementtypeid  latestperiodflag  fiscalchainseriesid  
0                  3                 0                    1  
1                  3                 0                    1  
2                  3                 0                    1  
3                  5                 0                    1  
4                  3                 0                    1

In [28]:
print ("Financial collection data : **ciqfincollectiondata**")
sql = """
select * from ciqfincollectiondata  limit 1000
"""
data = db.get_dataframe(sql)
data.head()

Financial collection data : **ciqfincollectiondata**


financialcollectionid  dataitemid  dataitemvalue  unittypeid  nmflag  \
0              139303258       41618        12.2749           0       0   
1              139303261       21842         2.3002           0       0   
2              139303261       21844         1.3590           0       0   
3              139303261       21845        40.5863           0       0   
4              139303261       21848        54.1414           0       0   

   pacvertofeedpop  
0              549  
1              549  
2              549  
3              549  
4              549

Financial collection data : **ciqfincollectiondata**


financialcollectionid  dataitemid  dataitemvalue  unittypeid  nmflag  \
0              139303258       41618        12.2749           0       0   
1              139303261       21842         2.3002           0       0   
2              139303261       21844         1.3590           0       0   
3              139303261       21845        40.5863           0       0   
4              139303261       21848        54.1414           0       0   

   pacvertofeedpop  
0              549  
1              549  
2              549  
3              549  
4              549

In [29]:
# print ("Financial collection data CashFlow: **ciqfincollectiondata**")
# sql = """
# select * from ciqfincollectiondata where 
# dataitemid in (select dataitemid from ciqdataitemfinancial where datacollectiontypeid = 2)

# limit 100
# """
# data = db.get_dataframe(sql)
# data.head()

In [30]:
print ("Financial collection data : **ciqfincollectiondata**")
sql = """
select * from ciqfincollectiondata limit 100
"""
data = db.get_dataframe(sql)
data.head()

Financial collection data : **ciqfincollectiondata**
Financial collection data : **ciqfincollectiondata**


financialcollectionid  dataitemid  dataitemvalue  unittypeid  nmflag  \
0              139303258       41618        12.2749           0       0   
1              139303261       21842         2.3002           0       0   
2              139303261       21844         1.3590           0       0   
3              139303261       21845        40.5863           0       0   
4              139303261       21848        54.1414           0       0   

   pacvertofeedpop  
0              549  
1              549  
2              549  
3              549  
4              549

financialcollectionid  dataitemid  dataitemvalue  unittypeid  nmflag  \
0              139303258       41618        12.2749           0       0   
1              139303261       21842         2.3002           0       0   
2              139303261       21844         1.3590           0       0   
3              139303261       21845        40.5863           0       0   
4              139303261       21848        54.1414           0       0   

   pacvertofeedpop  
0              549  
1              549  
2              549  
3              549  
4              549

In [31]:
print ("Financial financial collection : **ciqfincollection**")
sql = """
select * from ciqfincollection limit 100
"""
data = db.get_dataframe(sql)
data.head()

Financial financial collection : **ciqfincollection**


financialcollectionid  financialinstanceid  datacollectiontypeid  \
0                 284590               101548                     2   
1                 284594               250016                    12   
2                 284596               148279                     1   
3                 284597               176233                    15   
4                 284598               269925                    10   

   nummonths  currencyid  nonstandardlengthflag  
0          0         160                      0  
1         12         160                      0  
2         12         160                      0  
3          9         160                      0  
4         12         160                      0

Financial financial collection : **ciqfincollection**


financialcollectionid  financialinstanceid  datacollectiontypeid  \
0                 284590               101548                     2   
1                 284594               250016                    12   
2                 284596               148279                     1   
3                 284597               176233                    15   
4                 284598               269925                    10   

   nummonths  currencyid  nonstandardlengthflag  
0          0         160                      0  
1         12         160                      0  
2         12         160                      0  
3          9         160                      0  
4         12         160                      0

In [32]:
print ("Financial period detail type : **ciqfinperioddetailtype**")
sql = """
select * from ciqfinperioddetailtype limit 100
"""
data = db.get_dataframe(sql)
data.head()

Financial period detail type : **ciqfinperioddetailtype**
Financial period detail type : **ciqfinperioddetailtype**


perioddetailtypeid           perioddetailtypename
0                   1    Capital IQ Filing Date Data
1                   2  Compustat Collected Date Data

perioddetailtypeid           perioddetailtypename
0                   1    Capital IQ Filing Date Data
1                   2  Compustat Collected Date Data

In [33]:
print ("Financial  : **financialinstanceid**")
sql = """
select * from ciqfinancialinstance limit 100
"""
data = db.get_dataframe(sql)
data.head()

Financial  : **financialinstanceid**
Financial  : **financialinstanceid**


DatabaseError: Execution failed on sql '
select * from ciqfinancialinstance limit 100
': relation "ciqfinancialinstance" does not exist
LINE 2: select * from ciqfinancialinstance limit 100
                      ^


DatabaseError: Execution failed on sql '
select * from ciqfinancialinstance limit 100
': relation "ciqfinancialinstance" does not exist
LINE 2: select * from ciqfinancialinstance limit 100
                      ^


In [ ]:
print ("Financial period: **ciqfinperiod**")
sql = """
select a. from ciqfinperiod limit 100
"""
data = db.get_dataframe(sql)
data.head()

In [34]:
print("compensation")
sql = """

with test1 as (
select * from ciqcompensation limit 10
),
test2 as 
(select * from ciqcompensationtype limit 100)

select a.*, b.compensationtypename from test1 a, test2 b where a.compensationtypeid=b.compensationtypeid
"""

data = db.get_dataframe(sql)
data.head()

compensation
compensation


compensationid  proid filingdate  islatest  currencyid  \
0        10859765  65684 2004-12-31         0          55   
1        10859780  65684 2004-12-31         0          55   
2        10859785  65684 2004-12-31         0          55   
3        10859795  65684 2004-12-31         0          55   
4        10859806  65684 2006-04-03         0          55   

   compensationfiscalyear  compensationtypeid  compensationvalue  \
0                    2004                   1           265000.0   
1                    2004                   4                0.0   
2                    2004                   5                0.0   
3                    2004                   7                0.0   
4                    2005                  15           666808.0   

  periodenddate            compensationtypename  
0    2004-12-31                          Salary  
1    2004-12-31         Restricted Stock Awards  
2    2004-12-31                    Stock Grants  
3    2004-12-31        Long Term Incentive Plan  
4    2005-12-31  Total Annual Cash Compensation

compensationid  proid filingdate  islatest  currencyid  \
0        10859765  65684 2004-12-31         0          55   
1        10859780  65684 2004-12-31         0          55   
2        10859785  65684 2004-12-31         0          55   
3        10859795  65684 2004-12-31         0          55   
4        10859806  65684 2006-04-03         0          55   

   compensationfiscalyear  compensationtypeid  compensationvalue  \
0                    2004                   1           265000.0   
1                    2004                   4                0.0   
2                    2004                   5                0.0   
3                    2004                   7                0.0   
4                    2005                  15           666808.0   

  periodenddate            compensationtypename  
0    2004-12-31                          Salary  
1    2004-12-31         Restricted Stock Awards  
2    2004-12-31                    Stock Grants  
3    2004-12-31        Long Term Incentive Plan  
4    2005-12-31  Total Annual Cash Compensation

In [117]:
itemid = 28

sql = """
with fiscalYear as (
select * from ciqlatestinstancefinperiod where companyid = 24937 
AND fiscalyear >= EXTRACT(YEAR FROM now()) - 5 order by fiscalyear
),
itemName as (select dataitemid, dataitemname from 
    ciqdataitem 
    where dataitemid in ({itemid}
    ))

SELECT ti.tradingitemid,
    fd.dataitemvalue,
    fd.dataitemid,
    fd.financialperiodid,
    curr.isocode,
    c.companyid,
    itemName.dataitemname,
    unit.unittypevalue,
    fp.latestperiodflag,
    fiscalYear.fiscalyear
    
   FROM ciqcompany c
     JOIN ciqsecurity s ON c.companyid = s.companyid
     JOIN ciqtradingitem ti ON ti.securityid = s.securityid
     JOIN ciqlatestinstancefinperiod fp ON fp.companyid = c.companyid
     JOIN ciqperiodtype pt ON pt.periodtypeid = fp.periodtypeid
     JOIN ciqfinancialdata fd ON fd.financialperiodid = fp.financialperiodid
     JOIN ciqdataitem di ON di.dataitemid = fd.dataitemid
     JOIN ciqcurrency curr ON curr.currencyid = fp.currencyid
     JOIN fiscalYear ON fd.financialperiodid = fiscalYear.financialperiodid
     JOIN itemName on itemName.dataitemid = fd.dataitemid
     JOIN ciqfinunittype unit on unit.unittypeid = fd.unittypeid
  WHERE fp.periodtypeid = 1 AND c.companyid = 24937 
  AND fd.dataitemid = {itemid} 
  
  ORDER BY fd.dataitemid, fiscalYear.fiscalyear;
""".format(itemid=itemid)
data = db.get_dataframe(sql)
data.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0      297131480       233715.0          28         1761433273     USD   
1      238088778       233715.0          28         1761433273     USD   
2      237783682       233715.0          28         1761433273     USD   
3      418921286       233715.0          28         1761433273     USD   
4      607903026       233715.0          28         1761433273     USD   

   companyid    dataitemname  unittypevalue  latestperiodflag  fiscalyear  
0      24937  Total Revenues        1000000                 0        2015  
1      24937  Total Revenues        1000000                 0        2015  
2      24937  Total Revenues        1000000                 0        2015  
3      24937  Total Revenues        1000000                 0        2015  
4      24937  Total Revenues        1000000                 0        2015

tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0      297131480       233715.0          28         1761433273     USD   
1      238088778       233715.0          28         1761433273     USD   
2      237783682       233715.0          28         1761433273     USD   
3      418921286       233715.0          28         1761433273     USD   
4      607903026       233715.0          28         1761433273     USD   

   companyid    dataitemname  unittypevalue  latestperiodflag  fiscalyear  
0      24937  Total Revenues        1000000                 0        2015  
1      24937  Total Revenues        1000000                 0        2015  
2      24937  Total Revenues        1000000                 0        2015  
3      24937  Total Revenues        1000000                 0        2015  
4      24937  Total Revenues        1000000                 0        2015

In [123]:
data.groupby(["fiscalyear"]).mean()

tradingitemid  dataitemvalue  dataitemid  financialperiodid  \
fiscalyear                                                                
2015         4.212357e+08       233715.0        28.0       1.761433e+09   
2016         4.212357e+08       215639.0        28.0       1.814427e+09   
2017         4.212357e+08       229234.0        28.0       1.863764e+09   
2018         4.212357e+08       265595.0        28.0       1.917728e+09   
2019         4.212357e+08       260174.0        28.0       1.989628e+09   

            companyid  unittypevalue  latestperiodflag  
fiscalyear                                              
2015          24937.0      1000000.0               0.0  
2016          24937.0      1000000.0               0.0  
2017          24937.0      1000000.0               0.0  
2018          24937.0      1000000.0               0.0  
2019          24937.0      1000000.0               1.0

tradingitemid  dataitemvalue  dataitemid  financialperiodid  \
fiscalyear                                                                
2015         4.212357e+08       233715.0        28.0       1.761433e+09   
2016         4.212357e+08       215639.0        28.0       1.814427e+09   
2017         4.212357e+08       229234.0        28.0       1.863764e+09   
2018         4.212357e+08       265595.0        28.0       1.917728e+09   
2019         4.212357e+08       260174.0        28.0       1.989628e+09   

            companyid  unittypevalue  latestperiodflag  
fiscalyear                                              
2015          24937.0      1000000.0               0.0  
2016          24937.0      1000000.0               0.0  
2017          24937.0      1000000.0               0.0  
2018          24937.0      1000000.0               0.0  
2019          24937.0      1000000.0               1.0

In [125]:
for x in range(0, 5):
    print(x)

0
1
2
3
4
0
1
2
3
4


In [53]:
781 * 300 

234300

234300

In [79]:
print("compensation adjustment")
sql = """

WITH fiscalYear as (
        select * from ciqlatestinstancefinperiod where companyid = %s 
        AND fiscalyear >= EXTRACT(YEAR FROM now()) - 5 order by fiscalyear
        ),
        itemName as (select dataitemid, dataitemname from 
            ciqdataitem 
            where dataitemid in (4422, 
                                1008, 
                                1193, 
                                3028,  
                                3069, 
                                2169,
                                28,
                                4051,
                                1096
            ))
        
        SELECT ti.tradingitemid,
            fd.dataitemvalue,
            fd.dataitemid,
            fd.financialperiodid,
            curr.isocode,
            c.companyid,
            itemName.dataitemname,
            unit.unittypevalue,
            fp.latestperiodflag,
            fiscalYear.fiscalyear
            
           FROM ciqcompany c
             JOIN ciqsecurity s ON c.companyid = s.companyid
             JOIN ciqtradingitem ti ON ti.securityid = s.securityid
             JOIN ciqlatestinstancefinperiod fp ON fp.companyid = c.companyid
             JOIN ciqperiodtype pt ON pt.periodtypeid = fp.periodtypeid
             JOIN ciqfinancialdata fd ON fd.financialperiodid = fp.financialperiodid
             JOIN ciqdataitem di ON di.dataitemid = fd.dataitemid
             JOIN ciqcurrency curr ON curr.currencyid = fp.currencyid
             JOIN fiscalYear ON fd.financialperiodid = fiscalYear.financialperiodid
             JOIN itemName on itemName.dataitemid = fd.dataitemid
             JOIN ciqfinunittype unit on unit.unittypeid = fd.unittypeid
          WHERE fp.periodtypeid = 1 AND ti.tradingitemid = 2590360 
          AND fd.dataitemid in (4422, 
                                1008, 
                                1193, 
                                3028,  
                                3069, 
                                2169,
                                28,
                                4051,
                                1096)
          
          ORDER BY itemName.dataitemname;
"""

data = db.get_dataframe(sql, (24937,))
data.head()

compensation adjustment


tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0        2590360        48844.0        1096         1989628342     USD   
1        2590360        21120.0        1096         1761433273     USD   
2        2590360        25913.0        1096         1917727891     USD   
3        2590360        20289.0        1096         1863764179     USD   
4        2590360        20484.0        1096         1814427080     USD   

   companyid          dataitemname  unittypevalue  latestperiodflag  \
0      24937  Cash And Equivalents        1000000                 1   
1      24937  Cash And Equivalents        1000000                 0   
2      24937  Cash And Equivalents        1000000                 0   
3      24937  Cash And Equivalents        1000000                 0   
4      24937  Cash And Equivalents        1000000                 0   

   fiscalyear  
0        2019  
1        2015  
2        2018  
3        2017  
4        2016

compensation adjustment


tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
0        2590360        48844.0        1096         1989628342     USD   
1        2590360        21120.0        1096         1761433273     USD   
2        2590360        25913.0        1096         1917727891     USD   
3        2590360        20289.0        1096         1863764179     USD   
4        2590360        20484.0        1096         1814427080     USD   

   companyid          dataitemname  unittypevalue  latestperiodflag  \
0      24937  Cash And Equivalents        1000000                 1   
1      24937  Cash And Equivalents        1000000                 0   
2      24937  Cash And Equivalents        1000000                 0   
3      24937  Cash And Equivalents        1000000                 0   
4      24937  Cash And Equivalents        1000000                 0   

   fiscalyear  
0        2019  
1        2015  
2        2018  
3        2017  
4        2016

In [70]:
TOTAL_CURRENT_ASSET = 1008

total_asset = data[data["dataitemid"] == TOTAL_CURRENT_ASSET]
total_asset.head()

tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
10675      237783678       106869.0        1008         1814427080     USD   
10676      215778538       162819.0        1008         1989628342     USD   
10677      572470024       128645.0        1008         1863764179     USD   
10678      431237460       106869.0        1008         1814427080     USD   
10679      431237460        89378.0        1008         1761433273     USD   

       companyid          dataitemname  unittypevalue  latestperiodflag  \
10675      24937  Total Current Assets        1000000                 0   
10676      24937  Total Current Assets        1000000                 1   
10677      24937  Total Current Assets        1000000                 0   
10678      24937  Total Current Assets        1000000                 0   
10679      24937  Total Current Assets        1000000                 0   

       fiscalyear  
10675        2016  
10676        2019  
10677        2017  
10678        2016  
10679        2015

tradingitemid  dataitemvalue  dataitemid  financialperiodid isocode  \
10675      237783678       106869.0        1008         1814427080     USD   
10676      215778538       162819.0        1008         1989628342     USD   
10677      572470024       128645.0        1008         1863764179     USD   
10678      431237460       106869.0        1008         1814427080     USD   
10679      431237460        89378.0        1008         1761433273     USD   

       companyid          dataitemname  unittypevalue  latestperiodflag  \
10675      24937  Total Current Assets        1000000                 0   
10676      24937  Total Current Assets        1000000                 1   
10677      24937  Total Current Assets        1000000                 0   
10678      24937  Total Current Assets        1000000                 0   
10679      24937  Total Current Assets        1000000                 0   

       fiscalyear  
10675        2016  
10676        2019  
10677        2017  
10678        2016  
10679        2015

In [96]:
import datetime

SQL_DAILY_DATA = """
    SELECT  priceopen, pricehigh, pricelow,  priceclose,  volume, 
    extract (epoch from pricingdate) timeseries, pricingdate
	    FROM public.v_price_timeseries 
	    WHERE tradingitemid = %s AND pricingdate between %s and %s order by timeseries;
"""

now = datetime.datetime.now()
now_minus_year = datetime.datetime.now() - datetime.timedelta(days=365)

stock_data = db.get_dataframe(SQL_DAILY_DATA, (6179710, now_minus_year, now,))

stock_data.head()


priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  
0  2019-05-23  
1  2019-05-24  
2  2019-05-28  
3  2019-05-29  
4  2019-05-30

priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  
0  2019-05-23  
1  2019-05-24  
2  2019-05-28  
3  2019-05-29  
4  2019-05-30

In [97]:

def shifted_by(pd_series, row_shift):
    """
    get close price the day before shifted_by(data['close'], 1)
    :return: shifted_vals 
    """
    shifted_vals = pd_series.iloc[:-row_shift]
    shifted_vals.index = pd_series.index[row_shift:]
    return shifted_vals

In [98]:
stock_data.loc[:, 'prev_closed'] = shifted_by(stock_data['priceclose'], 1)
stock_data.head()

priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  prev_closed  
0  2019-05-23          NaN  
1  2019-05-24       282.14  
2  2019-05-28       282.78  
3  2019-05-29       280.15  
4  2019-05-30       278.27

priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  prev_closed  
0  2019-05-23          NaN  
1  2019-05-24       282.14  
2  2019-05-28       282.78  
3  2019-05-29       280.15  
4  2019-05-30       278.27

In [99]:
stock_data.loc[:, 'return'] = stock_data['priceclose'] - stock_data['prev_closed']
stock_data.head()

priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  prev_closed  return  
0  2019-05-23          NaN     NaN  
1  2019-05-24       282.14    0.64  
2  2019-05-28       282.78   -2.63  
3  2019-05-29       280.15   -1.88  
4  2019-05-30       278.27    0.76

priceopen  pricehigh  pricelow  priceclose     volume    timeseries  \
0     283.16     283.21   280.570      282.14   98733847  1.558570e+09   
1     283.74     284.20   282.090      282.78   55268095  1.558656e+09   
2     283.09     284.15   280.130      280.15   70029356  1.559002e+09   
3     278.91     279.36   276.710      278.27  104972927  1.559088e+09   
4     279.11     280.04   277.805      279.03   62523780  1.559174e+09   

  pricingdate  prev_closed  return  
0  2019-05-23          NaN     NaN  
1  2019-05-24       282.14    0.64  
2  2019-05-28       282.78   -2.63  
3  2019-05-29       280.15   -1.88  
4  2019-05-30       278.27    0.76

In [100]:
X = stock_data.loc[:, ["return"]].iloc[1:, :].values
X

array([[ 6.400e-01],
       [-2.630e+00],
       [-1.880e+00],
       [ 7.600e-01],
       [-3.760e+00],
       [-7.000e-01],
       [ 5.960e+00],
       [ 2.430e+00],
       [ 1.840e+00],
       [ 2.850e+00],
       [ 1.320e+00],
       [-7.000e-02],
       [-5.100e-01],
       [ 1.190e+00],
       [-3.200e-01],
       [ 1.100e-01],
       [ 3.030e+00],
       [ 6.600e-01],
       [ 2.800e+00],
       [-1.860e+00],
       [-3.600e-01],
       [-2.880e+00],
       [-2.900e-01],
       [ 1.030e+00],
       [ 1.500e+00],
       [ 2.660e+00],
       [ 7.700e-01],
       [ 2.370e+00],
       [-3.400e-01],
       [-1.640e+00],
       [ 3.700e-01],
       [ 1.420e+00],
       [ 7.000e-01],
       [ 1.340e+00],
       [ 1.000e-01],
       [-1.040e+00],
       [-1.970e+00],
       [ 1.090e+00],
       [-1.660e+00],
       [ 7.300e-01],
       [ 2.130e+00],
       [ 1.410e+00],
       [-1.440e+00],
       [ 2.010e+00],
       [-5.500e-01],
       [-7.400e-01],
       [-3.290e+00],
       [-2.59

array([[ 6.400e-01],
       [-2.630e+00],
       [-1.880e+00],
       [ 7.600e-01],
       [-3.760e+00],
       [-7.000e-01],
       [ 5.960e+00],
       [ 2.430e+00],
       [ 1.840e+00],
       [ 2.850e+00],
       [ 1.320e+00],
       [-7.000e-02],
       [-5.100e-01],
       [ 1.190e+00],
       [-3.200e-01],
       [ 1.100e-01],
       [ 3.030e+00],
       [ 6.600e-01],
       [ 2.800e+00],
       [-1.860e+00],
       [-3.600e-01],
       [-2.880e+00],
       [-2.900e-01],
       [ 1.030e+00],
       [ 1.500e+00],
       [ 2.660e+00],
       [ 7.700e-01],
       [ 2.370e+00],
       [-3.400e-01],
       [-1.640e+00],
       [ 3.700e-01],
       [ 1.420e+00],
       [ 7.000e-01],
       [ 1.340e+00],
       [ 1.000e-01],
       [-1.040e+00],
       [-1.970e+00],
       [ 1.090e+00],
       [-1.660e+00],
       [ 7.300e-01],
       [ 2.130e+00],
       [ 1.410e+00],
       [-1.440e+00],
       [ 2.010e+00],
       [-5.500e-01],
       [-7.400e-01],
       [-3.290e+00],
       [-2.59

In [101]:
stock_data.get("priceclose").iloc[1]

282.78

282.78

In [29]:
print("compensation adjustment")
sql = """

select * from ciqfinunittype limit 10
"""

data = db.get_dataframe(sql)
data.head()

compensation adjustment


unittypeid unittypename  unittypevalue
0           0       Actual              1
1           1    Thousands           1000
2           2     Millions        1000000

In [ ]:
print("compensation adjustment")
sql = """

with test1 as (
select * from ciqcompensationadjustment limit 10
),
test2 as 
(select * from ciqcompensationtype limit 100)

select a.*, b.compensationtypename from test1 a, test2 b where a.compensationtypeid=b.compensationtypeid
"""

data = db.get_dataframe(sql)
data.head()

In [36]:
print("compensation detail")
sql = """


select * from ciqcompensationdetail limit 10

"""

data = db.get_dataframe(sql)
data.head()

compensation detail


compensationdetailid     proid  compensationfiscalyear  \
0               4686047  35443237                    2007   
1               4686048  35443237                    2007   
2               4686049  35443237                    2007   
3               4686050  36308839                    2006   
4               4686052  36308839                    2006   

   compensationsubtypesequence  compensationsubtypeid compensationdetailvalue  \
0                            1                      4              2015-11-30   
1                            1                     14                  310046   
2                            1                     15                  138704   
3                            2                      2                       0   
4                            2                     12                       0   

        accessionnumber filingdate  relativeconstant  
0  0001193125-08-117789 2008-05-19               995  
1  0001193125-08-117789 2008-05-19               995  
2  0001193125-08-117789 2008-05-19               995  
3                  None 2008-04-24               988  
4                  None 2008-04-24               988

In [37]:
print("competitor")
sql = """


select * from ciqcompetitors limit 10

"""

data = db.get_dataframe(sql)
data.head()

competitor


competitorid  companyid  competitorcompanyid
0          4787    1876292              4494927
1          5166    4370561               325589
2          5556     325589              4370561
3          7615      35111               319092
4          7699      28914                27928

In [38]:
print("ciqdataitembankregulatory")
sql = """


select * from ciqdataitembankregulatory limit 10

"""

data = db.get_dataframe(sql)
data.head()

ciqdataitembankregulatory


dataitemid  datasetid  asreportedflag  unittypeid
0       30955          3               1           1
1       30955          6               1           1
2       30956          3               1           1
3       30956          6               1           1
4       30957          6               1           1

In [40]:
print("ciqdataitemtransaction")
sql = """


select * from ciqdataitemtransaction limit 10

"""

data = db.get_dataframe(sql)
data.head()

ciqdataitemtransaction


dataitemid  valuetypeid  objecttypeid
0      100504            2            14
1      100505            1            14
2      100506            1            14
3      100507            1            14
4      100508            1            14

In [41]:
print("dividend")
sql = """


select * from ciqdividend limit 10

"""

data = db.get_dataframe(sql)
data.head()

dividend


dividendid     exdate    paydate recorddate announceddate  divamount  \
0     7742561 2020-02-21 2020-03-16 2020-02-24    2020-01-15     0.8039   
1     7742562 2020-02-21 2020-02-28 2020-02-24    2020-02-20     0.0550   
2     7742563 2020-02-21 2020-02-28 2020-02-24    2020-02-20     0.1100   
3     7742564 2020-02-21 2020-02-28 2020-02-24    2020-02-20     0.0761   
4     7742565 2020-02-21 2020-02-28 2020-02-24    2020-02-20     0.0742   

   tradingitemid  exchangeid  currencyid  divfreqtypeid  dividendtypeid  \
0        6179254       117.0         160              1               1   
1      569872468       117.0         160              1               1   
2      571285487       117.0         160              1               1   
3      225219058       117.0         160              1               1   
4      428533026       117.0         160              1               1   

   supplementaltypeid netamount  grossamount  
0                 1.0      None          NaN  
1                 1.0      None          NaN  
2                 1.0      None          NaN  
3                 1.0      None          NaN  
4                 1.0      None          NaN

In [42]:
print("dividend")
sql = """


select * from ciqdividendfrequencytype limit 10

"""

data = db.get_dataframe(sql)
data.head()

dividend


divfreqtypeid  divfreqtypename
0              1          Monthly
1              2        Quarterly
2              3      Semi-Annual
3              4           Annual
4              5  Interim Payment

In [43]:
print("dividend payment")
sql = """


select * from ciqdividendpaymenttype limit 10

"""

data = db.get_dataframe(sql)
data.head()

dividend payment


dividendpaymenttypeid dividendpaymenttypename
0                      1                    Cash
1                      2                   Stock
2                      3                Spin-off
3                      4                  Rights
4                      5      Multiple Dividends

In [44]:
print("documents")
sql = """


select * from ciqdocument limit 10

"""

data = db.get_dataframe(sql)
data.head()

documents


documentid filingdate perioddate  documentsourceid  amendmentflag  \
0           1 1995-12-18        NaT                 0            NaN   
1           2 1995-12-18        NaT                 0            NaN   
2           5 1995-12-18        NaT                 0            NaN   
3           6 1995-11-15 1995-09-30                 0            0.0   
4           9 1995-11-15 1995-09-30                 0            0.0   

        accessionnumber  
0  0000000000-95-004501  
1  0000000000-95-004505  
2  0000000000-95-004508  
3  0000001952-95-000012  
4  0000001985-95-000004

In [45]:
print("documents")
sql = """


select * from ciqdocumentsourcetype limit 10

"""

data = db.get_dataframe(sql)
data.head()

documents


documentsourceid                documentsourcename
0                 0  EDGAR NGMS Dissemination Service
1                 1                   Company Website
2                 6                           Mergent
3                 7                             SEDAR
4                10               Independent Website

In [46]:
print("documents")
sql = """


select * from ciqdocumentformtype limit 10

"""

data = db.get_dataframe(sql)
data.head()

documents


documentformtypeid formtypename  \
0                 202       N-1A/A   
1                 203          N-2   
2                 204        N-2/A   
3                 205      N-23C-1   
4                 206    N-23C-1/A   

                                 formtypedescription  
0              Securities Registration (mutual fund)  
1  Securities Registration (close-end investment ...  
2  Securities Registration (close-end investment ...  
3  Reports by closed-end investment companies of ...  
4  Reports by closed-end investment companies of ...

In [48]:
print("documents")
sql = """


select * from ciqdocumentobjectreltype limit 10

"""

data = db.get_dataframe(sql)
data.head()

documents


documentobjectreltypeid relationshipdescription  objecttypeid
0                        1            Issuer/Filer             2
1                        2          Owner (Person)             3
2                        3         Owner (Company)             2
3                        4      Investment Manager             2
4                        5             Transaction            14

In [49]:
print("Event")
sql = """


select * from ciqevent limit 100

"""

data = db.get_dataframe(sql)
data.head()

Event


keydevid                                           headline  \
0   1005861  CPI Aerostructures Inc., Q2 2004 Earnings Call...   
1   1005862  Air Methods Corp., Q2 2004 Earnings Call, Aug-...   
2   1005864  Northcore Technologies, Inc., Q2 2004 Earnings...   
3   1005868  Electronics Boutique Holdings Corp., Q2 2005 E...   
4   1005869  Agile Software Corp., Q1 2005 Earnings Call, A...   

                                           situation mostimportantdateutc  \
0  CPI Aerostructures Inc., Q2 2004 Earnings Call...  2004-08-16 15:00:00   
1  Air Methods Corp., Q2 2004 Earnings Call, Aug ...  2004-08-12 20:15:00   
2  Northcore Technologies, Inc., Q2 2004 Earnings...  2004-08-12 14:00:00   
3  Electronics Boutique Holdings Corp., Q2 2005 E...  2004-08-19 21:00:00   
4  Agile Software Corp., Q1 2005 Earnings Call, A...  2004-08-19 21:00:00   

  announceddateutc         entereddate    lastmodifieddate  
0       2004-08-05 2004-08-06 13:33:00 2017-06-27 08:49:00  
1       2004-08-05 2004-08-06 13:33:00 2017-07-01 04:05:00  
2       2004-08-05 2004-08-06 13:34:00 2017-07-01 04:05:00  
3       2004-08-05 2004-08-06 13:36:00 2017-07-01 04:05:00  
4       2004-08-05 2004-08-06 13:37:00 2017-07-01 04:05:00

In [ ]:
sys.path.append("from numpy import npv
import numpy
import datetime
import requests
from bs4 import BeautifulSoup
import decimal
from decimal import Decimal
import time

class DCFModel:
    def __init__(self):
        items = []
        data = self.pull_data('DGS5')[0]
        for line in data:
            item = {
                'date': line['date'],
                'values': {'value': line['value']},
                'timestamp': Decimal(str(time.mktime(
                    datetime.datetime.strptime(line['date'], "%Y-%m-%d").timetuple())))
            }
            items.append(item)
        self.risk_free = items[-1]['values']['value']


    def load_values(self):
        print("Risk Free Rate: {}".format(self.risk_free))

        query = accessor.query_dynamodb('AAPL', 2017)
        beta_mktrtn = calculateBeta_and_mktReturn('AAPL')
        print("Beta: {}".format(beta_mktrtn[0]))
        print("Market Return: {}".format(beta_mktrtn[1]))

        equity = float(query[0]['totalequity'])
        print("Total Equity: {}".format(equity))
        debt = float(query[0]['debt'])
        print("Total Debt: {}".format(debt))
        interest = float(query[0]['cashinterestpaid'])
        print("Total Interest: {}".format(interest))
        tax_rate = float(query[0]['efftaxrate'])
        print("Tax Rate: {}".format(tax_rate))
        fcf = float(query[0]['freecashflow'])
        print("Free Cash Flow: {}".format(fcf))
        shrs_outstanding = float(query[0]['weightedavebasicsharesos'])
        print("Shares Outstanding: {}".format(shrs_outstanding))
        fcf_growth = float(query[0]['fcffgrowth'])
        print("Free Cash Flow Growth: {}".format(fcf_growth))
        ebitda = float(query[0]['ebitda'])
        print("EBITDA: {}".format(ebitda))
        ev = float(query[0]['enterprisevalue'])
        print("Enterprise Value: {}".format(ev))

        ke = calculate_capm(risk_free, beta_mktrtn[0], beta_mktrtn[1])
        print("Cost of Equity: {}".format(ke))
        kd = calculate_kd(interest, debt)
        print("Cost of Debt: {}".format(kd))
        wacc = calculate_wacc(equity, debt, ke, kd, tax_rate)
        print("WACC: {}".format(wacc))
        dcf = calculateDCF(fcf, fcf_growth, wacc, shrs_outstanding, ebitda, ev)
        print("DCF Analysis Predicted Stock Price: '${:,.2f}'".format(dcf))

        return dcf


    def pull_data(self, symbol):
        url = "https://fred.stlouisfed.org/data/" + symbol + ".txt"
        hdr = {'User-Agent': 'we dont want to wait for more requests /u/m&j'}
        response = requests.get(url, stream=True, headers=hdr)
        soup = BeautifulSoup(response.text, "html.parser")
        split_txt = soup.text.splitlines()
        meta = True
        meta_dict = {}
        meta_dict['notes'] = ''
        data_dict = {}
        for el in split_txt:
            try:
                if meta:
                    if '' == el:
                        pass
                    elif ' ' == el[0]:
                        meta_dict['notes'] += el.replace('                     ', ' ')
                    elif 'DATE' in el:
                        meta = False
                    elif 'Series' in el[:10] or 'Seasonal' in el[:10] or 'Date Range' in el[:10] or 'Last' in el[:10]:
                        tab_split = el.split(None, 2)
                        tab_split = [tab_split[0] + '_' + tab_split[1], tab_split[2]]
                        meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                    else:
                        tab_split = el.split(None, 1)
                        if len(tab_split) == 1:
                            meta_dict[tab_split[0].lower().replace(':', '')] = ''
                        else:
                            meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                else:
                    tab_split = el.split(None, 1)
                    data_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
            except IndexError as e:
                pass

        format_data = []
        for k, v in data_dict.items():
            if v == '.':
                pass
            # format_data.append({'date': k, 'value': decimal.Decimal('NaN')})
            else:
                format_data.append({'date': k, 'value': float(v)})
        return format_data, meta_dict

    # aka cost of equity
    def calculate_capm(self, risk_free_rate, beta, market_return):
        return risk_free_rate + (beta * (market_return - risk_free_rate))

    def calculate_kd(self, tot_int, tot_debt):
        return tot_int/tot_debt

    def calculate_wacc(self, equity, debt, cost_of_equity, cost_of_debt, corporate_tax_rate):
        v = equity + debt
        return ((equity/v) * cost_of_equity) + ((debt/v) * cost_of_debt * (1 - corporate_tax_rate))

    def calculateDCF(self, current_fcf, growth_rate, WACC, shares_outstanding, ebitda, ev):
        fcf_list = []
        for x in range(0, 5):
            current_fcf = current_fcf*(1+growth_rate)
            fcf_list.append(current_fcf)
            # print(current_fcf)
        # print(fcf_list)
        # print(npv(WACC,fcf_list))
        pre_terminal_value = npv(WACC, fcf_list)
        terminal_value = ebitda * (ev/ebitda)
        return (pre_terminal_value + terminal_value)/shares_outstanding

    def calculateBeta_and_mktReturn(self, ticker):
        now = datetime.datetime.now()
        now_minus_year = datetime.datetime.now() - datetime.timedelta(days=365)
        stock = quandl.get('EOD/{}'.format(ticker), column_index='4', start_date=now_minus_year, end_date=now.date())
        market = quandl.get('EOD/SPY', column_index='4', start_date=now_minus_year, end_date=now.date())
        # market = pd.read_csv('SPY.csv')
        # print(market)
        # print(market.get('Close'))

        stock_lst = []
        mkt_lst = []
        for x in range(len(stock)):
            stock_lst.append(float(stock.get('Close')[x]))
            mkt_lst.append(float(market.get('Close')[x]))

        stock_return_list = []
        market_return_list = []
        for y in range(1, len(stock)):
            stock_return_list.append((stock_lst[y]-stock_lst[y-1])/stock_lst[y-1])
            market_return_list.append((mkt_lst[y] - mkt_lst[y - 1]) / mkt_lst[y - 1])

        # print(stock_return_list)
        # print(market_return_list)
        # print(lst)
        # stock_rtn = (stock_lst[-1] - stock_lst[0]) / (stock_lst[0])
        # print(stock_rtn)
        #
        mkt_rtn = (mkt_lst[-1] - mkt_lst[0]) / (mkt_lst[0])
        # print(mkt_rtn)

        Covariance = numpy.cov(stock_return_list, market_return_list, bias=True)[0][1]
        # print(Covariance)
        variance_mkt = numpy.var(market_return_list)
        # print(variance_mkt)
        beta = (Covariance / variance_mkt)

        return [beta, mkt_rtn]


# this test is for DHI... WACC is almost perfect based on my calculations
if __name__ == '__main__':
    dcf = DCFModel()
    dcf.load_values()
")

In [5]:
from numpy import npv
import numpy
import datetime
import requests
from bs4 import BeautifulSoup
import decimal
from decimal import Decimal
import time

In [6]:
from numpy import npv
import numpy
import datetime
import requests
from bs4 import BeautifulSoup
import decimal
from decimal import Decimal
import time

class DCFModel:
    def __init__(self):
        items = []
        data = self.pull_data('DGS5')[0]
        for line in data:
            item = {
                'date': line['date'],
                'values': {'value': line['value']},
                'timestamp': Decimal(str(time.mktime(
                    datetime.datetime.strptime(line['date'], "%Y-%m-%d").timetuple())))
            }
            items.append(item)
        self.risk_free = items[-1]['values']['value']


    def load_values(self):
        print("Risk Free Rate: {}".format(self.risk_free))

        query = accessor.query_dynamodb('AAPL', 2017)
        beta_mktrtn = calculateBeta_and_mktReturn('AAPL')
        print("Beta: {}".format(beta_mktrtn[0]))
        print("Market Return: {}".format(beta_mktrtn[1]))

        equity = float(query[0]['totalequity'])
        print("Total Equity: {}".format(equity))
        debt = float(query[0]['debt'])
        print("Total Debt: {}".format(debt))
        interest = float(query[0]['cashinterestpaid'])
        print("Total Interest: {}".format(interest))
        tax_rate = float(query[0]['efftaxrate'])
        print("Tax Rate: {}".format(tax_rate))
        fcf = float(query[0]['freecashflow'])
        print("Free Cash Flow: {}".format(fcf))
        shrs_outstanding = float(query[0]['weightedavebasicsharesos'])
        print("Shares Outstanding: {}".format(shrs_outstanding))
        fcf_growth = float(query[0]['fcffgrowth'])
        print("Free Cash Flow Growth: {}".format(fcf_growth))
        ebitda = float(query[0]['ebitda'])
        print("EBITDA: {}".format(ebitda))
        ev = float(query[0]['enterprisevalue'])
        print("Enterprise Value: {}".format(ev))

        ke = calculate_capm(risk_free, beta_mktrtn[0], beta_mktrtn[1])
        print("Cost of Equity: {}".format(ke))
        kd = calculate_kd(interest, debt)
        print("Cost of Debt: {}".format(kd))
        wacc = calculate_wacc(equity, debt, ke, kd, tax_rate)
        print("WACC: {}".format(wacc))
        dcf = calculateDCF(fcf, fcf_growth, wacc, shrs_outstanding, ebitda, ev)
        print("DCF Analysis Predicted Stock Price: '${:,.2f}'".format(dcf))

        return dcf


    def pull_data(self, symbol):
        url = "https://fred.stlouisfed.org/data/" + symbol + ".txt"
        hdr = {'User-Agent': 'we dont want to wait for more requests /u/m&j'}
        response = requests.get(url, stream=True, headers=hdr)
        soup = BeautifulSoup(response.text, "html.parser")
        split_txt = soup.text.splitlines()
        meta = True
        meta_dict = {}
        meta_dict['notes'] = ''
        data_dict = {}
        for el in split_txt:
            try:
                if meta:
                    if '' == el:
                        pass
                    elif ' ' == el[0]:
                        meta_dict['notes'] += el.replace('                     ', ' ')
                    elif 'DATE' in el:
                        meta = False
                    elif 'Series' in el[:10] or 'Seasonal' in el[:10] or 'Date Range' in el[:10] or 'Last' in el[:10]:
                        tab_split = el.split(None, 2)
                        tab_split = [tab_split[0] + '_' + tab_split[1], tab_split[2]]
                        meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                    else:
                        tab_split = el.split(None, 1)
                        if len(tab_split) == 1:
                            meta_dict[tab_split[0].lower().replace(':', '')] = ''
                        else:
                            meta_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
                else:
                    tab_split = el.split(None, 1)
                    data_dict[tab_split[0].lower().replace(':', '')] = tab_split[1]
            except IndexError as e:
                pass

        format_data = []
        for k, v in data_dict.items():
            if v == '.':
                pass
            # format_data.append({'date': k, 'value': decimal.Decimal('NaN')})
            else:
                format_data.append({'date': k, 'value': float(v)})
        return format_data, meta_dict

    # aka cost of equity
    def calculate_capm(self, risk_free_rate, beta, market_return):
        return risk_free_rate + (beta * (market_return - risk_free_rate))

    def calculate_kd(self, tot_int, tot_debt):
        return tot_int/tot_debt

    def calculate_wacc(self, equity, debt, cost_of_equity, cost_of_debt, corporate_tax_rate):
        v = equity + debt
        return ((equity/v) * cost_of_equity) + ((debt/v) * cost_of_debt * (1 - corporate_tax_rate))

    def calculateDCF(self, current_fcf, growth_rate, WACC, shares_outstanding, ebitda, ev):
        fcf_list = []
        for x in range(0, 5):
            current_fcf = current_fcf*(1+growth_rate)
            fcf_list.append(current_fcf)
            # print(current_fcf)
        # print(fcf_list)
        # print(npv(WACC,fcf_list))
        pre_terminal_value = npv(WACC, fcf_list)
        terminal_value = ebitda * (ev/ebitda)
        return (pre_terminal_value + terminal_value)/shares_outstanding

    def calculateBeta_and_mktReturn(self, ticker):
        now = datetime.datetime.now()
        now_minus_year = datetime.datetime.now() - datetime.timedelta(days=365)
        stock = quandl.get('EOD/{}'.format(ticker), column_index='4', start_date=now_minus_year, end_date=now.date())
        market = quandl.get('EOD/SPY', column_index='4', start_date=now_minus_year, end_date=now.date())
        # market = pd.read_csv('SPY.csv')
        # print(market)
        # print(market.get('Close'))

        stock_lst = []
        mkt_lst = []
        for x in range(len(stock)):
            stock_lst.append(float(stock.get('Close')[x]))
            mkt_lst.append(float(market.get('Close')[x]))

        stock_return_list = []
        market_return_list = []
        for y in range(1, len(stock)):
            stock_return_list.append((stock_lst[y]-stock_lst[y-1])/stock_lst[y-1])
            market_return_list.append((mkt_lst[y] - mkt_lst[y - 1]) / mkt_lst[y - 1])

        # print(stock_return_list)
        # print(market_return_list)
        # print(lst)
        # stock_rtn = (stock_lst[-1] - stock_lst[0]) / (stock_lst[0])
        # print(stock_rtn)
        #
        mkt_rtn = (mkt_lst[-1] - mkt_lst[0]) / (mkt_lst[0])
        # print(mkt_rtn)

        Covariance = numpy.cov(stock_return_list, market_return_list, bias=True)[0][1]
        # print(Covariance)
        variance_mkt = numpy.var(market_return_list)
        # print(variance_mkt)
        beta = (Covariance / variance_mkt)

        return [beta, mkt_rtn]


# this test is for DHI... WACC is almost perfect based on my calculations
if __name__ == '__main__':
    dcf = DCFModel()
    dcf.load_values()


OverflowError: mktime argument out of range